# [CptS 215 Data Analytics Systems and Algorithms](https://piazza.com/wsu/fall2017/cpts215/home)
[Washington State University](https://wsu.edu)

[Gina Sprint](http://eecs.wsu.edu/~gsprint/)
# L9-2 Binary Search Trees

Learner objectives for this lesson:
* Review binary search trees
* Implement a map ADT using a binary search tree
* Perform algorithm analysis of binary search trees


## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Miller and Ranum](http://interactivepython.org/runestone/static/pythonds/index.html)
* [Dr. Ananth Kalyanaraman](http://www.eecs.wsu.edu/~ananth/)'s CptS 223 notes

## Binary Search Trees
Based on how we insert and retrieve items from a binary tree, we can implement a binary tree as a map abstract data type. This is called a binary search tree. A binary search tree is based on the *BST property*: keys (data) that are less than the parent key are found in the left subtree, keys that are greater than the parent key are found in the right subtree.

Example:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Binary_search_tree.svg/2000px-Binary_search_tree.svg.png" width="300">
(image from [https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Binary_search_tree.svg/2000px-Binary_search_tree.svg.png](https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Binary_search_tree.svg/2000px-Binary_search_tree.svg.png))

### BST Traversal
Performing an in-order traversal of a BST will visit each node in sorted order ($\mathcal{O}(n)$):
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bst_inorder.png" width="200">

### BST Insertion
To insert a new node into a BST, we start at the root and do the following:
* If new node == current node, we have a duplicate, handle accordingly
* If new node < current node
    * If current node has no left child, insert new node as current node's left child
    * Else insert new node into current node's left subtree (recursive call)
* If new node > current node
    * If current node has no right child, insert the node as current node's right child
    * Else insert new node into current node's right subtree (recursive call)
    
For example, suppose we want to insert 5 into the following BST:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bst_insert.png" width="600">

### BST Deletion
There are three cases for removal:
1. Case 1: Node to remove has 0 children
     * Action: Just remove it and update the parent's reference to None
1. Case 2: Node to remove has 1 child
    * Action: Just remove it and update the parent's reference to the node's child
    * Case 1 and 2 example:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bst_remove_case1.png" width="600">
1. Case 3: Node to remove has 2 children
    * Action: Replace the node with successor. Remove the successor (case 1 or 2 above).
    * Example:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bst_remove_case2.png" width="600">

## BST Implementation

In [1]:
class BSTNode:
    '''
    
    '''
    def __init__(self, key, val, left=None, right=None, parent=None):
        '''
        
        '''
        self.key = key
        self.value = val
        self.left_child = left
        self.right_child = right
        self.parent = parent
        
    def __iter__(self):
        '''
        Yield freezes the state of the function so that the next time the function 
        is called it continues executing from the exact point it left off earlier.
        '''
        if self:
            if self.has_left_child():
                 for elem in self.left_child:
                    yield elem
            yield self.key
            if self.has_right_child():
                 for elem in self.right_child:
                    yield elem

    def has_left_child(self):
        '''
        
        '''
        return self.left_child

    def has_right_child(self):
        '''
        
        '''
        return self.right_child

    def is_left_child(self):
        '''
        
        '''
        return self.parent and self.parent.left_child == self

    def is_right_child(self):
        '''
        
        '''
        return self.parent and self.parent.right_child == self

    def is_root(self):
        '''
        
        '''
        return not self.parent

    def is_leaf(self):
        '''
        
        '''
        return not (self.right_child or self.left_child)

    def has_any_children(self):
        '''
        
        '''
        return self.right_child or self.left_child

    def has_both_children(self):
        '''
        
        '''
        return self.right_child and self.left_child

    def replace_node_data(self, key, value, lc, rc):
        '''
        Overwrite this node's information with new information in the parameters
        '''
        self.key = key
        self.value = value
        self.left_child = lc
        self.right_child = rc
        if self.has_left_child():
            self.left_child.parent = self
        if self.has_right_child():
            self.right_child.parent = self
            
    def splice_out(self):
        '''
        Go directly to the node we want to splice out and makes the right changes.
        Handles case 1 and 2 of delete()
        No need to search for node to delete (unlike delete())
        '''
        if self.is_leaf(): # delete() case 1 (no children)
            if self.is_left_child():
                self.parent.left_child = None
            else:
                self.parent.right_child = None
        elif self.has_any_children(): # delete case 2 (exactly one child)
            if self.has_left_child():
                if self.is_left_child():
                    self.parent.left_child = self.left_child
                else:
                    self.parent.right_child = self.left_child
                self.left_child.parent = self.parent
            else:
                if self.is_left_child():
                    self.parent.left_child = self.right_child
                else:
                    self.parent.right_child = self.right_child
                self.right_child.parent = self.parent

    def find_successor(self):
        '''
        3 cases when looking for the successor:
        
        1. If the node has a right child, then the successor is the smallest key in the right subtree.
        2. If the node has no right child and is the left child of its parent, then the parent is the successor.
        3. If the node is the right child of its parent, and itself has no right child, 
        then the successor to this node is the successor of its parent, excluding this node.
        '''
        succ = None
        if self.has_right_child():
            succ = self.right_child.find_min()
        else:
            if self.parent:
                if self.is_left_child():
                    succ = self.parent
                else:
                    self.parent.right_child = None
                    succ = self.parent.find_successor()
                    self.parent.right_child = self
        return succ

    def find_min(self):
        '''
        Find the minimum key in a subtree.
        Left-most child in the subtree.
        '''
        current = self
        while current.has_left_child():
            current = current.left_child
        return current
            
class BinarySearchTree:
    '''
    
    '''
    def __init__(self):
        '''
        
        '''
        self.root = None
        self.size = 0

    def length(self):
        '''
        
        '''
        return self.size

    def __len__(self):
        '''
        Return the number of key-value pairs stored in the map.
        '''
        return self.size

    def __iter__(self):
        '''
        
        '''
        return self.root.__iter__()
    
    def __setitem__(self, k, v):
        '''
        
        '''
        self.put(k,v)
        
    def __getitem__(self, key):
        '''
        
        '''
        return self.get(key)
    
    def __delitem__(self,key):
        '''
        Delete the key-value pair from the map using a statement of the form del map[key].
        '''
        self.delete(key)
    
    def __contains__(self, key):
        '''
        Return True for a statement of the form key in map, if the given key is in the map.
        '''
        if self._get(key, self.root):
            return True
        else:
            return False
    
    def put(self, key, val):
        '''
        Add a new key-value pair to the map. 
        If the key is already in the map then replace the old value with the new value.
        
        If there is not a root then put will create a new BSTNode and install it as the root of the tree. 
        If a root node is already in place then put calls the private, recursive, helper function _put to search the tree
        '''
        if self.root:
            self._put(key, val, self.root)
        else:
            self.root = BSTNode(key, val)
        self.size = self.size + 1

    def _put(self, key, val, current_node):
        '''
        Starting at the root of the tree, search the binary tree comparing the new key 
        to the key in the current node. If the new key is less than the current node, 
        search the left subtree. If the new key is greater than the current node, 
        search the right subtree.

        When there is no left (or right) child to search, we have found the position 
        in the tree where the new node should be installed.

        To add a node to the tree, create a new BStNode object and insert the object 
        at the point discovered in the previous step.
        '''
        if key == current_node.key: # handle duplicates by updating value
            current_node.value = val
        elif key < current_node.key:
            if current_node.has_left_child():
                self._put(key, val, current_node.left_child)
            else:
                current_node.left_child = BSTNode(key, val, parent=current_node)
        else:
            if current_node.has_right_child():
                self._put(key, val, current_node.right_child)
            else:
                current_node.right_child = BSTNode(key, val, parent=current_node)
                    
    def get(self, key):
        '''
        Given a key, return the value stored in the map or None otherwise.
        '''
        if self.root:
            res = self._get(key, self.root)
            if res:
                   return res.value
            else:
                   return None
        else:
            return None

    def _get(self, key, current_node):
        '''
        searches the tree recursively until it gets to a non-matching leaf node or finds a matching key. 
        When a matching key is found, the value stored in the payload of the node is returned.
        '''
        if not current_node:
            return None
        elif current_node.key == key:
            return current_node
        elif key < current_node.key:
            return self._get(key, current_node.left_child)
        else:
            return self._get(key, current_node.right_child)

    def delete(self, key):
        '''
        Find the node to delete by searching the tree using the 
        _get method to find the BSTNode that needs to be removed. 
        
        If the tree only has a single node, that means we are removing the root of the tree, 
        but we still must check to make sure the key of the root matches the key that is to be deleted.
        '''
        if self.size > 1:
            node_to_remove = self._get(key, self.root)
            if node_to_remove:
                self.remove(node_to_remove)
                self.size = self.size - 1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def remove(self, current_node):
        '''
        3 cases to consider:
        1. The node to be deleted has no children
        --->Delete the node and remove the reference to this node in the parent.
        2. The node to be deleted has only one child
        -->Promote the child to take the place of its parent.
        -->If the current node has no parent, it must be the root. Replace the key, value, left_child, 
        and right_child data by calling the replace_node_data method on the root.
        3. The node to be deleted has two children
        -->Search the tree for a node (successor) that can be used to replace the one scheduled for deletion
        -->Remove the successor and put it in the tree in place of the node to be deleted.
        '''
        if current_node.is_leaf(): #leaf (case 1)
            if current_node == current_node.parent.left_child:
                current_node.parent.left_child = None
            else:
                current_node.parent.right_child = None
        elif current_node.has_both_children(): #interior (case 3)
            succ = current_node.find_successor()
            succ.splice_out()
            current_node.key = succ.key
            current_node.value = succ.value
        else: # this node has one child (case 2)
            if current_node.has_left_child():
                if current_node.is_left_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.left_child
                elif current_node.is_right_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.left_child
                else: # current_node is root
                    current_node.replace_node_data(current_node.left_child.key,
                                    current_node.left_child.value,
                                    current_node.left_child.left_child,
                                    current_node.left_child.right_child)
            else:
                if current_node.is_left_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.right_child
                elif current_node.is_right_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.right_child
                else: # current_node is root
                    current_node.replace_node_data(current_node.right_child.key,
                                    current_node.right_child.value,
                                    current_node.right_child.left_child,
                                    current_node.right_child.right_child)
    def in_order_traversal(self):
        '''
        
        '''
        if self.size > 0:
            self.in_order_helper(self.root)
            print()
        else:
            print("Empty tree")
            
    def in_order_helper(self, current_node):
        '''
        
        '''
        if current_node is not None:
            self.in_order_helper(current_node.left_child)
            print(str(current_node.key) + ":" + str(current_node.value), end=" ")
            self.in_order_helper(current_node.right_child)


mytree = BinarySearchTree()
mytree[3] = "red"
mytree[4] = "blue"
mytree[6] = "yellow"
mytree[2] = "at"

print(mytree[6])
print(mytree[2])
mytree.in_order_traversal()

# check duplicate key handling
mytree[4] = "green"
mytree.in_order_traversal()

# use of __iter__()
for node in mytree:
    print(node)
    
# check delete
del mytree[3]
mytree.in_order_traversal()

del mytree[2]
mytree.in_order_traversal()

yellow
at
2:at 3:red 4:blue 6:yellow 
2:at 3:red 4:green 6:yellow 
2
3
4
6
2:at 4:green 6:yellow 
4:green 6:yellow 


## BST Analysis
### Insertion (`put()`)
* In a perfectly balanced binary tree (same number of nodes in the left subtree and the right subtree of the root), the worst case performance is based on the number of nodes in the tree ($n$) and the height of the tree ($log_{2}(n)$):  $\mathcal{O}(log_{2}(n))$. The maximum number of comparisons needed to search for the proper place to insert a new node is $log_{2}(n)$. 
* In a non-perfectly balanced binary tree, the worst case performance is based on the number of nodes in the tree ($n$): $\mathcal{O}(n)$. Imagine inserting nodes in sorted order. In this case, the tree would actually look like a list! The maximum number of comparisons needed to search for the proper place to insert a node would be the same as linear search.
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bst_worst_case.png" width="400">

### Retrieval (`get()`), Delete (`del`), Find Min/Max
* Best and average case (perfectly balanced binary tree): $\mathcal{O}(log_{2}(n))$
* Worst case (imperfectly balanced binary tree): $\mathcal{O}(n)$

## Practice Problems
Note: the following problems are adapted from Koffman and Wolfgang.

### 1
Show the BST that would be formed for the following data items. Exchange the first and last items in each list, and rebuild the tree that would be formed if the items were inserted in the new order.
1. "happy", "depressed", "manic", "sad", "ecstatic"
1. 45, 30, 15, 50, 60, 20, 25, 90

### 2
Write a `main()` function to thoroughly test a BST.